In [1]:
! pip install  onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 79.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.2 MB/s eta 0:00:00


In [25]:
import onnxruntime as ort
import numpy as np
from transformers import DistilBertTokenizer,DistilBertForSequenceClassification
import torch
import time

In [3]:
distil_model = DistilBertForSequenceClassification.from_pretrained(
    "ashish-001/DistilBert-Amazon-review-sentiment-classifier")
distil_tokenizer = DistilBertTokenizer.from_pretrained(
    "ashish-001/DistilBert-Amazon-review-sentiment-classifier")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
session = ort.InferenceSession("/kaggle/input/distilbert_onnc_quantized_for_amazon_review/onnx/default/1/distilbertmodel_quantized.onnx", providers=["CPUExecutionProvider"])

In [ ]:
text=distil_tokenizer("hello",return_tensors='np')
input_names = [inp.name for inp in session.get_inputs()]
onnx_inputs = {name: text[name] for name in text}
outputs = session.run(None, onnx_inputs)
prob=torch.nn.functional.sigmoid(logits)
ind=np.argmax([probs[0][0].item(),probs[0][1].item()])
print("Positive" if ind else  "Negative")

In [17]:
t=distil_tokenizer("This is a  good product",max_length=256,
        truncation=True,
        padding="max_length",
        return_tensors="pt")
output = distil_model(**t)
logits = output.logits
probs = torch.nn.functional.sigmoid(logits)
ind=np.argmax([probs[0][0].item(),probs[0][1].item()])
print("Positive" if ind else  "Negative")

tensor([[0.2150, 0.8316]], grad_fn=<SigmoidBackward0>)


In [27]:
text_input = "This is a good product"


num_runs = 10 


onnx_times = []
for _ in range(num_runs):
    start_time = time.time()
    text = distil_tokenizer(text_input, return_tensors="np")
    input_names = [inp.name for inp in session.get_inputs()]
    onnx_inputs = {name: text[name] for name in text}
    outputs = session.run(None, onnx_inputs)
    logits = torch.tensor(outputs[0])  
    probs = torch.nn.functional.sigmoid(logits)
    onnx_times.append(time.time() - start_time)

onnx_avg_time = sum(onnx_times) / num_runs
print(f"ONNX Average Inference Time: {onnx_avg_time:.6f} sec")


pytorch_times = []
for _ in range(num_runs):
    start_time = time.time()
    t = distil_tokenizer(
        text_input,
        max_length=256,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    output = distil_model(**t)
    logits = output.logits
    probs = torch.nn.functional.sigmoid(logits)
    pytorch_times.append(time.time() - start_time)

pytorch_avg_time = sum(pytorch_times) / num_runs
print(f"PyTorch Average Inference Time: {pytorch_avg_time:.6f} sec")


speedup = (pytorch_avg_time - onnx_avg_time) / pytorch_avg_time * 100
print(f"ONNX is {speedup:.2f}% faster than PyTorch" if speedup > 0 else f"PyTorch is {-speedup:.2f}% faster than ONNX")

ONNX Average Inference Time: 0.007902 sec
PyTorch Average Inference Time: 0.214982 sec
ONNX is 96.32% faster than PyTorch
